In [1]:
# Dependencies and Setup

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import sem 


In [2]:
# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Combine the data into a single dataset
mouse_drug_data= "mouse_drug_data.csv"
clinical_trial_data = "clinicaltrial_data.csv"

mouse_drug_df = pd.read_csv(mouse_drug_data)

clinical_trial_df = pd.read_csv(clinical_trial_data)

combined_trial_df = pd.merge(mouse_drug_df, clinical_trial_df, how='outer', on='Mouse ID')

    

In [4]:
# Display the data table for preview
combined_trial_df

,Mouse ID,Drug,Timepoint,Tumor Volume (mm3),Metastatic Sites
0,f234,Stelasyn,0,45.000000,0
1,f234,Stelasyn,5,47.313491,0
2,f234,Stelasyn,10,47.904324,0
3,f234,Stelasyn,15,48.735197,1
4,f234,Stelasyn,20,51.112713,2
...,...,...,...,...,...
1901,j235,Placebo,0,45.000000,0
1902,j235,Placebo,5,46.588820,0
1903,j235,Placebo,10,48.018788,1
1904,j235,Placebo,15,51.161138,1


# Tumor Response to Treatment


In [5]:
# Store the Mean Tumor Volume Data Grouped by Drug and Timepoint 
grouped_trial_df = combined_trial_df[["Drug", "Timepoint", "Tumor Volume (mm3)"]]

#Drug, Timepoint, Tumor Volume (mm3)

In [6]:
# Convert to DataFrame
grouped_trial_df 

# Preview DataFrame

,Drug,Timepoint,Tumor Volume (mm3)
0,Stelasyn,0,45.000000
1,Stelasyn,5,47.313491
2,Stelasyn,10,47.904324
3,Stelasyn,15,48.735197
4,Stelasyn,20,51.112713
...,...,...,...
1901,Placebo,0,45.000000
1902,Placebo,5,46.588820
1903,Placebo,10,48.018788
1904,Placebo,15,51.161138


In [66]:
#tumor_volume_na=tumor_renamed["Tumor(mm3)"].sem(axis=1)
tumor_volume_na=tumor_renamed.groupby("Drug").sem()

tumor_volume_na

,Timepoint,Tumor(mm3)
Drug,,
Capomulin,0.948518,0.329346
Ceftamin,1.070629,0.469821
Infubinol,1.009879,0.483691
Ketapril,1.023238,0.603860
Naftisol,1.040081,0.596466
Placebo,1.032495,0.581331
Propriva,1.061776,0.512884
Ramicane,0.945433,0.320955
Stelasyn,0.983582,0.543853


In [85]:
Drug=grouped_trial_df["Drug"]

Timepoint=grouped_trial_df["Timepoint"]

Tumor=grouped_trial_df["Tumor Volume (mm3)"].sem(axis=0,skipna=True)



#df.sem(axis = 1, skipna = False) 

#Tumor = grouped_trial_df["Tumor Volume (mm3)"].map"Tumor Volume (mm3)"].sem())

#print(data['salary'].std())

standard_error_tumor=pd.DataFrame({"Drug":Drug,
                    "Timepoint":Timepoint,
                    "Tumor Volume (mm3)":Tumor})

standard_error_tumor

#clear=standard_error_tumor.dropna(subset=["Tumor Volume (mm3)"])

#clear.head()

,Drug,Timepoint,Tumor Volume (mm3)
0,Stelasyn,0,0.203161
1,Stelasyn,5,0.203161
2,Stelasyn,10,0.203161
3,Stelasyn,15,0.203161
4,Stelasyn,20,0.203161
...,...,...,...
1901,Placebo,0,0.203161
1902,Placebo,5,0.203161
1903,Placebo,10,0.203161
1904,Placebo,15,0.203161


In [24]:
# Convert to DataFrame

# Preview DataFrame


In [25]:
#Creating a scatter plot that shows how the tumor volume changes over time for each treatment.



In [ ]:
#Creating a scatter plot that shows how the number of metastatic (cancer spreading) sites changes over time for each treatment.


In [ ]:
#Creating a scatter plot that shows the number of mice still alive through the course of treatment (Survival Rate)



In [ ]:
#Creating a bar graph that compares the total % tumor volume change for each drug across the full 45 days.


In [ ]:
#Include 3 observations about the results of the study. Use the visualizations you generated from the study data as the basis for your observations.
